In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/MyDrive/FinancialPhraseBank-v1.0.zip"

Archive:  /content/gdrive/MyDrive/FinancialPhraseBank-v1.0.zip
   creating: FinancialPhraseBank-v1.0/
  inflating: FinancialPhraseBank-v1.0/License.txt  
   creating: __MACOSX/
   creating: __MACOSX/FinancialPhraseBank-v1.0/
  inflating: __MACOSX/FinancialPhraseBank-v1.0/._License.txt  
  inflating: FinancialPhraseBank-v1.0/README.txt  
  inflating: __MACOSX/FinancialPhraseBank-v1.0/._README.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_50Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_66Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_75Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_AllAgree.txt  


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [ ]:
data = pd.read_csv('/content/FinancialPhraseBank-v1.0/Sentences_50Agree.txt', sep='@', names=['text','label'], engine='python', encoding='latin1')

train, test = train_test_split(data, test_size=0.2, random_state=0)
#train, valid = train_test_split(train, test_size=0.1, random_state=0)

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
#valid = valid.reset_index(drop=True)

In [ ]:
train_aug = pd.read_csv('/content/gdrive/MyDrive/train_gpt4o_augmented_filtered_2505.csv')
gpt4 = pd.read_csv('/content/gdrive/MyDrive/gpt4o_generated_text_2405.csv')
forex = pd.read_csv('/content/gdrive/MyDrive/sentiment_annotated_with_texts.csv')

In [ ]:
forex = forex[['text','true_sentiment']].copy()
forex.rename(columns={'true_sentiment': 'label'}, inplace=True)
forex['label'] = forex['label'].str.lower()
forex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2291 non-null   object
 1   label   2291 non-null   object
dtypes: object(2)
memory usage: 35.9+ KB


In [ ]:
train_aug = train_aug[['text','label']].copy()
train_aug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1001 non-null   object
 1   label   1001 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [ ]:
gpt4 = gpt4[['text','label']].copy()
gpt4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    419 non-null    object
 1   label   419 non-null    object
dtypes: object(2)
memory usage: 6.7+ KB


In [ ]:
train_augmented = pd.concat([train, train_aug, gpt4], axis=0)
train_augmented.reset_index(drop=True, inplace=True)
train_augmented = train_augmented.sample(frac=1).reset_index(drop=True).copy()
train_augmented = train_augmented.sample(frac=1).reset_index(drop=True).copy()
train_augmented.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5296 entries, 0 to 5295
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5296 non-null   object
 1   label   5296 non-null   object
dtypes: object(2)
memory usage: 82.9+ KB


In [ ]:
train_augmented, valid_augmented = train_test_split(train_augmented, test_size=0.1, random_state=0)
print(train_augmented.info())
print(valid_augmented.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4766 entries, 130 to 2732
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4766 non-null   object
 1   label   4766 non-null   object
dtypes: object(2)
memory usage: 111.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 530 entries, 5069 to 1075
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    530 non-null    object
 1   label   530 non-null    object
dtypes: object(2)
memory usage: 12.4+ KB
None


In [ ]:
# List of variables to keep
variables_to_keep = ['train_augmented', 'valid_augmented', 'test', 'forex']

# Get the current global variables
current_globals = globals().copy()

# Delete variables that are not in the variables_to_keep list
for var_name in current_globals:
    if var_name not in variables_to_keep and not var_name.startswith('__'):
        del globals()[var_name]

# Verify remaining variables
print(globals().keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '__', '___', 'test', 'forex', 'train_augmented', 'valid_augmented', 'current_globals', 'var_name'])


In [ ]:
!pip install transformers[torch] accelerate -U
!pip install datasets
import torch
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, get_scheduler
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Define the label list
label_list = ['positive', 'negative', 'neutral']
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

# Load the pre-trained tokenizer and model
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))
model.config.hidden_dropout_prob = 0.1

# Load your datasets
train_dataset = Dataset.from_pandas(train_augmented)
valid_dataset = Dataset.from_pandas(valid_augmented)
test_dataset = Dataset.from_pandas(test)
forex_dataset = Dataset.from_pandas(forex)

del globals()['train_augmented']
del globals()['valid_augmented']
del globals()['test']
del globals()['forex']

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=64)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
forex_dataset = forex_dataset.map(tokenize_function, batched=True)

# Encode the labels
def encode_labels(examples):
    examples['label'] = [label_to_id[label] for label in examples['label']]
    return examples

train_dataset = train_dataset.map(encode_labels, batched=True)
valid_dataset = valid_dataset.map(encode_labels, batched=True)
test_dataset = test_dataset.map(encode_labels, batched=True)
forex_dataset = forex_dataset.map(encode_labels, batched=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 23.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

...

Map:   0%|          | 0/2291 [00:00<?, ? examples/s]

Map:   0%|          | 0/4766 [00:00<?, ? examples/s]

Map:   0%|          | 0/530 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

Map:   0%|          | 0/2291 [00:00<?, ? examples/s]

In [ ]:
from torch.optim.lr_scheduler import LambdaLR

# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
forex_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': acc,  # Primary metric
        'f1': f1,
    }

# Define layer-wise learning rate decay
def get_optimizer_grouped_parameters(model, base_learning_rate, layerwise_decay):
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = []
    layers = [model.base_model.embeddings] + list(model.base_model.encoder.layer)

    for i, layer in enumerate(layers):
        lr = base_learning_rate * (layerwise_decay ** (len(layers) - i - 1))
        optimizer_grouped_parameters.extend([
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.01,
                "lr": lr,
            },
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": lr,
            },
        ])
    optimizer_grouped_parameters.extend([
        {
            "params": [p for n, p in model.classifier.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.01,
            "lr": base_learning_rate,
        },
        {
            "params": [p for n, p in model.classifier.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
            "lr": base_learning_rate,
        },
    ])
    return optimizer_grouped_parameters

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/gdrive/MyDrive/results',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=6,
    weight_decay=0.01,
    warmup_ratio=0.2,
    #save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# Set up custom optimizer and learning rate scheduler with layer-wise learning rate decay
base_learning_rate = 2e-5
layerwise_decay = 0.95  # Decay learning rate by 5% per layer
optimizer_grouped_parameters = get_optimizer_grouped_parameters(model, base_learning_rate, layerwise_decay)
optimizer = torch.optim.AdamW(optimizer_grouped_parameters)

# Calculate the number of training steps
num_training_steps = training_args.num_train_epochs * len(train_dataset) // training_args.per_device_train_batch_size
num_warmup_steps = int(num_training_steps * training_args.warmup_ratio)

# Define the slanted triangular learning rate scheduler
def get_slanted_triangular_scheduler(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0,
            (num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    return LambdaLR(optimizer, lr_lambda)

# Set up the learning rate scheduler
lr_scheduler = get_slanted_triangular_scheduler(optimizer, num_warmup_steps, num_training_steps)

# Custom trainer to handle layer unfreezing
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.freeze_all_layers()

    def freeze_all_layers(self):
        for param in self.model.base_model.parameters():
            param.requires_grad = False
        for param in self.model.classifier.parameters():
            param.requires_grad = True

    def unfreeze_next_layer(self):
        for name, param in self.model.named_parameters():
            if not param.requires_grad:
                param.requires_grad = True
                break

    def training_step(self, *args, **kwargs):
        # Unfreeze next layer every third of an epoch
        global_step = self.state.global_step
        total_steps = self.state.max_steps
        if global_step > 0 and global_step % (total_steps // (3 * training_args.num_train_epochs)) == 0:
            self.unfreeze_next_layer()
        return super().training_step(*args, **kwargs)

# Instantiate CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),
)

In [ ]:
# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("/content/gdrive/MyDrive/finetuned_finbert_25052024_v6_GPT4o_rephrased_and_GPT_DFT_STL")

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.565732,0.788679,0.794149
2,No log,0.510796,0.800000,0.804649
3,No log,0.486146,0.800000,0.804429
4,No log,0.470335,0.803774,0.807769
5,No log,0.455003,0.803774,0.807313
6,No log,0.449971,0.805660,0.809019
